In [1]:
!pip install accelerate torch gradio transformers git+https://github.com/huggingface/diffusers sentencepiece opencv-python


  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-imzeq8kh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-imzeq8kh
  Resolved https://github.com/huggingface/diffusers to commit 914a585be8187ec0ad92fab4f072c992f8c297cd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 19.8 MB/s eta 0:00:00


In [1]:
# Install required packages
#!pip install accelerate torch gradio transformers git+https://github.com/huggingface/diffusers sentencepiece opencv-python

import os

# Define a fallback for environments without GPU
if os.environ.get("SPACES_ZERO_GPU") is not None:
    import spaces
else:
    class spaces:
        @staticmethod
        def GPU(func):
            def wrapper(*args, **kwargs):
                return func(*args, **kwargs)
            return wrapper

import torch
from diffusers import DiffusionPipeline
from diffusers.utils import export_to_video
import gradio as gr

# Application configuration
TITLE = "AI Video Generator 🌟"
DESCRIPTION = """\
🌈 Transform your imagination into stunning videos using advanced AI technology with Mochi-1-preview.\
Experience the magic of generative art! 🎥
"""
BUY_ME_A_COFFEE = """
<a href="https://buymeacoffee.com/ruslanmv" target="_blank">
    <button style="background-color: #FFDD00; border: none; color: black;
                    padding: 10px 20px; text-align: center;
                    text-decoration: none; display: inline-block;
                    font-size: 16px; margin: 4px 2px; cursor: pointer;
                    border-radius: 10px;">\
        ☕ Buy Me a Coffee
    </button>
</a>
"""
MODEL_PRE_TRAINED_ID = "genmo/mochi-1-preview"
EXAMPLES = [
    [
        "A colossal griffin perched atop a crumbling gothic castle, its golden wings outstretched against a blood-red sunset.  Below, a raging battle between knights and goblins unfolds amidst the ruins of a once-great city.  The air is filled with the clash of steel, the cries of the wounded, and the roar of the griffin's echoing cry.",
        90,
        30,
    ],
    [
        "A sprawling cyberpunk metropolis illuminated by holographic advertisements, flying vehicles zipping between neon skyscrapers, and augmented reality overlays blurring the line between the physical and digital world. The streets teem with diverse crowds of people with cybernetic enhancements, while the constant hum of technology fills the air.",
        70,
        24,
    ],
    [
        "An enchanted meadow where unicorns graze among glowing wildflowers. \"\
        Wisps of light float in the air, and a sparkling waterfall cascades into \"\
        a crystal-clear pond surrounded by colorful butterflies.",
        60,
        25,
    ],
    [
        "A sprawling underwater utopia with bioluminescent architecture, giant \"\
        jellyfish drifting gracefully, and schools of exotic fish weaving \"\
        through coral tunnels. The city is alive with vibrant marine life.",
        80,
        30,
    ],
    [
        "A vast alien desert with shimmering sands of gold and silver, \"\
        punctuated by colossal crystal spires. Twin suns set in the distance, \"\
        casting long, surreal shadows across the dunes.",
        75,
        28,
    ],
]

# Load the pre-trained model
pipe = DiffusionPipeline.from_pretrained(
    MODEL_PRE_TRAINED_ID, variant="bf16", torch_dtype=torch.bfloat16
)

# Enable memory-saving optimizations
pipe.enable_model_cpu_offload()
pipe.enable_vae_tiling()

def generate_video(prompt, num_frames=84, fps=30, high_quality=False):
    """Generate a video based on the input prompt."""
    if high_quality:
        print("High quality option selected. Requires 42GB VRAM.")
        if os.environ.get("SPACES_ZERO_GPU") is not None:
            raise RuntimeError("High quality option may fail on ZeroGPU environments.")
        with torch.autocast("cuda", torch.bfloat16, cache_enabled=False):
            frames = pipe(prompt, num_frames=num_frames).frames[0]
    else:
        print("Standard quality option selected.")
        frames = pipe(prompt, num_frames=num_frames).frames[0]

    video_path = "generated_video.mp4"
    export_to_video(frames, video_path, fps=fps)
    return video_path

# Define the Gradio interface
interface = gr.Interface(
    fn=generate_video,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter a vivid text prompt... 🔍"),
        gr.Slider(minimum=1, maximum=240, value=84, label="Frames 🎥"),
        gr.Slider(minimum=1, maximum=60, value=30, label="FPS (Frames Per Second) ⏱"),
        gr.Checkbox(label="High Quality (Requires 42GB VRAM) 🛠"),
    ],
    outputs=gr.Video(label="Generated Video"),
    title=TITLE,
    description=DESCRIPTION,
    examples=EXAMPLES,
    article=BUY_ME_A_COFFEE,
)

# Apply custom CSS for better alignment
interface.css = """
.interface-title {
    text-align: center;
    font-size: 2em;
    color: #4A90E2;
    font-family: 'Arial', sans-serif;
}
.interface-description {
    text-align: center;
    font-size: 1.2em;
    color: #333333;
    margin-bottom: 20px;
}
"""

# Launch the Gradio application
if __name__ == "__main__":
    interface.launch()



A mixture of bf16 and non-bf16 filenames will be loaded.
Loaded bf16 filenames:
[transformer/diffusion_pytorch_model.bf16-00003-of-00003.safetensors, vae/diffusion_pytorch_model.bf16.safetensors, transformer/diffusion_pytorch_model.bf16-00001-of-00003.safetensors, transformer/diffusion_pytorch_model.bf16-00002-of-00003.safetensors]
Loaded non-bf16 filenames:
[text_encoder/model-00001-of-00002.safetensors, text_encoder/model-00002-of-00002.safetensors
If this behavior is not expected, please check your folder structure.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af23e68baddbaf7bcc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
